In [1]:
from auto_login import get_curr_path,get_credentials,auto_login,generate_access_token
from macd import compute_ema
import os 
import logging 
from kiteconnect import KiteConnect
import datetime as dt
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np 
import ta

In [2]:
folder_name = 'api_keys'
curr_dir = get_curr_path(folder_name)

file_name = 'credentials.txt'
api_key,api_secret,user_name,pwd,totp_key = get_credentials(curr_dir,file_name)

request_token = auto_login(api_key,user_name,pwd,totp_key)

access_token = generate_access_token(request_token,api_key,api_secret)

In [3]:
kite = KiteConnect(api_key=api_key)
kite.set_access_token(access_token=access_token)

In [4]:
instrument_dump = kite.instruments('NSE')
instrument_df = pd.DataFrame(instrument_dump)
instrument_df.to_csv('NSE_instruments.csv',index=False)

In [5]:
instrument_df.head()

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,256265,1001,NIFTY 50,NIFTY 50,0.0,,0.0,0.0,0,EQ,INDICES,NSE
1,256777,1003,NIFTY MIDCAP 100,NIFTY MIDCAP 100,0.0,,0.0,0.0,0,EQ,INDICES,NSE
2,260105,1016,NIFTY BANK,NIFTY BANK,0.0,,0.0,0.0,0,EQ,INDICES,NSE
3,260617,1018,NIFTY 100,NIFTY 100,0.0,,0.0,0.0,0,EQ,INDICES,NSE
4,257033,1004,NIFTY DIV OPPS 50,NIFTY DIV OPPS 50,0.0,,0.0,0.0,0,EQ,INDICES,NSE


In [6]:
def instrumentLookup(instrument_df,symbol):
    """Looks up instrument token for a given script from instrument dump"""
    try:
        return instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1
    
def fetchOHLC(ticker,interval,duration):
    """extracts historical data and outputs in the form of dataframe"""
    instrument = instrumentLookup(instrument_df,ticker)
    data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration), dt.date.today(),interval))
    data.set_index("date",inplace=True)
    return data

In [7]:
infy_df = fetchOHLC('INFY','5minute',5)
infy_df.to_csv('infy.csv')

In [8]:
infy_df.head()

,open,high,low,close,volume
date,,,,,
2024-06-03 09:15:00+05:30,1440.00,1440.00,1422.50,1426.05,608537
2024-06-03 09:20:00+05:30,1425.35,1426.70,1418.45,1421.45,334479
2024-06-03 09:25:00+05:30,1421.85,1422.25,1415.75,1418.85,150139
2024-06-03 09:30:00+05:30,1419.00,1419.15,1413.05,1414.50,153774
2024-06-03 09:35:00+05:30,1414.55,1415.05,1410.60,1410.80,197865


In [9]:
df = infy_df.copy()

In [10]:
# def compute_super_trend

In [11]:
df['hl2'] = (df['high'] + df['low'])/2

In [12]:
df.head()

,open,high,low,close,volume,hl2
date,,,,,,
2024-06-03 09:15:00+05:30,1440.00,1440.00,1422.50,1426.05,608537,1431.250
2024-06-03 09:20:00+05:30,1425.35,1426.70,1418.45,1421.45,334479,1422.575
2024-06-03 09:25:00+05:30,1421.85,1422.25,1415.75,1418.85,150139,1419.000
2024-06-03 09:30:00+05:30,1419.00,1419.15,1413.05,1414.50,153774,1416.100
2024-06-03 09:35:00+05:30,1414.55,1415.05,1410.60,1410.80,197865,1412.825


In [13]:
df['tr'] = np.maximum.reduce([df['high'] - df['low'],
                  abs(df['high'] - df['close'].shift(1)),
                  abs(df['low'] - df['close'].shift(1))])
df.head()

,open,high,low,close,volume,hl2,tr
date,,,,,,,
2024-06-03 09:15:00+05:30,1440.00,1440.00,1422.50,1426.05,608537,1431.250,NaN
2024-06-03 09:20:00+05:30,1425.35,1426.70,1418.45,1421.45,334479,1422.575,8.25
2024-06-03 09:25:00+05:30,1421.85,1422.25,1415.75,1418.85,150139,1419.000,6.50
2024-06-03 09:30:00+05:30,1419.00,1419.15,1413.05,1414.50,153774,1416.100,6.10
2024-06-03 09:35:00+05:30,1414.55,1415.05,1410.60,1410.80,197865,1412.825,4.45


In [14]:
# assume n = 7 for supertrend
n = 7
alpha = 1/n

df.loc[df.index[n-1], 'atr'] = df['tr'][:n].mean()
df.head(7)

,open,high,low,close,volume,hl2,tr,atr
date,,,,,,,,
2024-06-03 09:15:00+05:30,1440.00,1440.00,1422.50,1426.05,608537,1431.250,NaN,NaN
2024-06-03 09:20:00+05:30,1425.35,1426.70,1418.45,1421.45,334479,1422.575,8.25,NaN
2024-06-03 09:25:00+05:30,1421.85,1422.25,1415.75,1418.85,150139,1419.000,6.50,NaN
2024-06-03 09:30:00+05:30,1419.00,1419.15,1413.05,1414.50,153774,1416.100,6.10,NaN
2024-06-03 09:35:00+05:30,1414.55,1415.05,1410.60,1410.80,197865,1412.825,4.45,NaN
2024-06-03 09:40:00+05:30,1410.65,1413.40,1410.50,1411.55,109628,1411.950,2.90,NaN
2024-06-03 09:45:00+05:30,1411.35,1411.50,1405.70,1411.30,213022,1408.600,5.85,5.675


In [15]:
for i in range(n, len(df)):
    df.loc[df.index[i], 'atr'] = alpha * df.loc[df.index[i], 'tr'] + (1 - alpha) * df.loc[df.index[i-1], 'atr']


In [16]:
df.head(10)

,open,high,low,close,volume,hl2,tr,atr
date,,,,,,,,
2024-06-03 09:15:00+05:30,1440.00,1440.00,1422.50,1426.05,608537,1431.250,NaN,NaN
2024-06-03 09:20:00+05:30,1425.35,1426.70,1418.45,1421.45,334479,1422.575,8.25,NaN
2024-06-03 09:25:00+05:30,1421.85,1422.25,1415.75,1418.85,150139,1419.000,6.50,NaN
2024-06-03 09:30:00+05:30,1419.00,1419.15,1413.05,1414.50,153774,1416.100,6.10,NaN
2024-06-03 09:35:00+05:30,1414.55,1415.05,1410.60,1410.80,197865,1412.825,4.45,NaN
2024-06-03 09:40:00+05:30,1410.65,1413.40,1410.50,1411.55,109628,1411.950,2.90,NaN
2024-06-03 09:45:00+05:30,1411.35,1411.50,1405.70,1411.30,213022,1408.600,5.85,5.675000
2024-06-03 09:50:00+05:30,1411.20,1412.95,1410.30,1412.30,100998,1411.625,2.65,5.242857
2024-06-03 09:55:00+05:30,1412.20,1416.00,1411.70,1415.50,98795,1413.850,4.30,5.108163


In [17]:
multiplier = 3 
df['basic_ub'] = df['hl2'] + (multiplier*df['atr'])
df['basic_lb'] = df['hl2'] - (multiplier*df['atr'])
df.head(10)

,open,high,low,close,volume,hl2,tr,atr,basic_ub,basic_lb
date,,,,,,,,,,
2024-06-03 09:15:00+05:30,1440.00,1440.00,1422.50,1426.05,608537,1431.250,NaN,NaN,NaN,NaN
2024-06-03 09:20:00+05:30,1425.35,1426.70,1418.45,1421.45,334479,1422.575,8.25,NaN,NaN,NaN
2024-06-03 09:25:00+05:30,1421.85,1422.25,1415.75,1418.85,150139,1419.000,6.50,NaN,NaN,NaN
2024-06-03 09:30:00+05:30,1419.00,1419.15,1413.05,1414.50,153774,1416.100,6.10,NaN,NaN,NaN
2024-06-03 09:35:00+05:30,1414.55,1415.05,1410.60,1410.80,197865,1412.825,4.45,NaN,NaN,NaN
2024-06-03 09:40:00+05:30,1410.65,1413.40,1410.50,1411.55,109628,1411.950,2.90,NaN,NaN,NaN
2024-06-03 09:45:00+05:30,1411.35,1411.50,1405.70,1411.30,213022,1408.600,5.85,5.675000,1425.625000,1391.575000
2024-06-03 09:50:00+05:30,1411.20,1412.95,1410.30,1412.30,100998,1411.625,2.65,5.242857,1427.353571,1395.896429
2024-06-03 09:55:00+05:30,1412.20,1416.00,1411.70,1415.50,98795,1413.850,4.30,5.108163,1429.174490,1398.525510


In [18]:
df.loc[df.index[n-1], 'ub'] = df.loc[df.index[n-1],'basic_ub'] 
df.loc[df.index[n-1], 'lb'] = df.loc[df.index[n-1],'basic_lb'] 

df.head(10)


,open,high,low,close,volume,hl2,tr,atr,basic_ub,basic_lb,ub,lb
date,,,,,,,,,,,,
2024-06-03 09:15:00+05:30,1440.00,1440.00,1422.50,1426.05,608537,1431.250,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-03 09:20:00+05:30,1425.35,1426.70,1418.45,1421.45,334479,1422.575,8.25,NaN,NaN,NaN,NaN,NaN
2024-06-03 09:25:00+05:30,1421.85,1422.25,1415.75,1418.85,150139,1419.000,6.50,NaN,NaN,NaN,NaN,NaN
2024-06-03 09:30:00+05:30,1419.00,1419.15,1413.05,1414.50,153774,1416.100,6.10,NaN,NaN,NaN,NaN,NaN
2024-06-03 09:35:00+05:30,1414.55,1415.05,1410.60,1410.80,197865,1412.825,4.45,NaN,NaN,NaN,NaN,NaN
2024-06-03 09:40:00+05:30,1410.65,1413.40,1410.50,1411.55,109628,1411.950,2.90,NaN,NaN,NaN,NaN,NaN
2024-06-03 09:45:00+05:30,1411.35,1411.50,1405.70,1411.30,213022,1408.600,5.85,5.675000,1425.625000,1391.575000,1425.625,1391.575
2024-06-03 09:50:00+05:30,1411.20,1412.95,1410.30,1412.30,100998,1411.625,2.65,5.242857,1427.353571,1395.896429,NaN,NaN
2024-06-03 09:55:00+05:30,1412.20,1416.00,1411.70,1415.50,98795,1413.850,4.30,5.108163,1429.174490,1398.525510,NaN,NaN


In [19]:
for i in range(n,len(df)):
    if ((df.loc[df.index[i],'basic_ub'] < df.loc[df.index[i-1],'ub']) or 
    (df.loc[df.index[i-1],'close'] > df.loc[df.index[i-1],'ub'])):
        df.loc[df.index[i],'ub'] = df.loc[df.index[i],'basic_ub']
    else:
        df.loc[df.index[i],'ub'] = df.loc[df.index[i-1],'ub']
    
    if ((df.loc[df.index[i],'basic_lb'] > df.loc[df.index[i-1],'lb']) or 
    (df.loc[df.index[i-1],'close'] < df.loc[df.index[i-1],'ub'])):
        df.loc[df.index[i],'lb'] = df.loc[df.index[i],'basic_lb']
    else:
        df.loc[df.index[i],'lb'] = df.loc[df.index[i-1],'lb']

df
    


,open,high,low,close,volume,hl2,tr,atr,basic_ub,basic_lb,ub,lb
date,,,,,,,,,,,,
2024-06-03 09:15:00+05:30,1440.00,1440.00,1422.50,1426.05,608537,1431.250,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-03 09:20:00+05:30,1425.35,1426.70,1418.45,1421.45,334479,1422.575,8.25,NaN,NaN,NaN,NaN,NaN
2024-06-03 09:25:00+05:30,1421.85,1422.25,1415.75,1418.85,150139,1419.000,6.50,NaN,NaN,NaN,NaN,NaN
2024-06-03 09:30:00+05:30,1419.00,1419.15,1413.05,1414.50,153774,1416.100,6.10,NaN,NaN,NaN,NaN,NaN
2024-06-03 09:35:00+05:30,1414.55,1415.05,1410.60,1410.80,197865,1412.825,4.45,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-07 13:30:00+05:30,1531.30,1532.10,1530.00,1531.10,154639,1531.050,2.10,2.798700,1539.446099,1522.653901,1539.446099,1522.653901
2024-06-07 13:35:00+05:30,1531.10,1534.45,1531.00,1533.35,129119,1532.725,3.45,2.891742,1541.400227,1524.049773,1539.446099,1524.049773
2024-06-07 13:40:00+05:30,1533.35,1534.10,1531.60,1532.70,218945,1532.850,2.50,2.835779,1541.357338,1524.342662,1539.446099,1524.342662


In [20]:
isUpTrend,isDownTrend = 'up','down'
init_trend_dir = isDownTrend

In [21]:
df['trend_direction'] = init_trend_dir

In [22]:
df['super_trend'] = pd.Series(dtype=float)

In [23]:
for i in range(len(df)):
    if i < n:
        df.loc[df.index[i], 'trend_direction'] = init_trend_dir
    else:
        prev_super_trend = df.loc[df.index[i-1], 'super_trend'] if i > n else None
        prev_upper_band = df.loc[df.index[i-1], 'ub']
        prev_lower_band = df.loc[df.index[i-1], 'lb']
        
        if prev_super_trend == prev_upper_band:
            if df.loc[df.index[i], 'close'] > df.loc[df.index[i], 'ub']:
                df.loc[df.index[i], 'trend_direction'] = isUpTrend
            else:
                df.loc[df.index[i], 'trend_direction'] = isDownTrend
        else:
            if df.loc[df.index[i], 'close'] < df.loc[df.index[i], 'lb']:
                df.loc[df.index[i], 'trend_direction'] = isDownTrend
            else:
                df.loc[df.index[i], 'trend_direction'] = isUpTrend
        
        # Set the super trend value based on the trend direction
        if df.loc[df.index[i], 'trend_direction'] == isUpTrend:
            df.loc[df.index[i], 'super_trend'] = df.loc[df.index[i], 'lb']
        else:
            df.loc[df.index[i], 'super_trend'] = df.loc[df.index[i], 'ub']


In [24]:
df

,open,high,low,close,volume,hl2,tr,atr,basic_ub,basic_lb,ub,lb,trend_direction,super_trend
date,,,,,,,,,,,,,,
2024-06-03 09:15:00+05:30,1440.00,1440.00,1422.50,1426.05,608537,1431.250,NaN,NaN,NaN,NaN,NaN,NaN,down,NaN
2024-06-03 09:20:00+05:30,1425.35,1426.70,1418.45,1421.45,334479,1422.575,8.25,NaN,NaN,NaN,NaN,NaN,down,NaN
2024-06-03 09:25:00+05:30,1421.85,1422.25,1415.75,1418.85,150139,1419.000,6.50,NaN,NaN,NaN,NaN,NaN,down,NaN
2024-06-03 09:30:00+05:30,1419.00,1419.15,1413.05,1414.50,153774,1416.100,6.10,NaN,NaN,NaN,NaN,NaN,down,NaN
2024-06-03 09:35:00+05:30,1414.55,1415.05,1410.60,1410.80,197865,1412.825,4.45,NaN,NaN,NaN,NaN,NaN,down,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-07 13:30:00+05:30,1531.30,1532.10,1530.00,1531.10,154639,1531.050,2.10,2.798700,1539.446099,1522.653901,1539.446099,1522.653901,up,1522.653901
2024-06-07 13:35:00+05:30,1531.10,1534.45,1531.00,1533.35,129119,1532.725,3.45,2.891742,1541.400227,1524.049773,1539.446099,1524.049773,up,1524.049773
2024-06-07 13:40:00+05:30,1533.35,1534.10,1531.60,1532.70,218945,1532.850,2.50,2.835779,1541.357338,1524.342662,1539.446099,1524.342662,up,1524.342662


In [25]:
new_df = infy_df.copy()

In [26]:
def compute_super_trend(df,n,multiplier):
    df['hl2'] = (df['high'] + df['low'])/2
    df['tr'] = np.maximum.reduce([df['high'] - df['low'],
                  abs(df['high'] - df['close'].shift(1)),
                  abs(df['low'] - df['close'].shift(1))])
    alpha = 1/n
    
    df.loc[df.index[n-1], 'atr'] = df['tr'][:n].mean()
    for i in range(n, len(df)):
        df.loc[df.index[i], 'atr'] = alpha * df.loc[df.index[i], 'tr'] + (1 - alpha) * df.loc[df.index[i-1], 'atr']

    df['basic_ub'] = df['hl2'] + (multiplier*df['atr'])
    df['basic_lb'] = df['hl2'] - (multiplier*df['atr'])
    df.loc[df.index[n-1], 'ub'] = df.loc[df.index[n-1],'basic_ub'] 
    df.loc[df.index[n-1], 'lb'] = df.loc[df.index[n-1],'basic_lb'] 

    for i in range(n,len(df)):
        if ((df.loc[df.index[i],'basic_ub'] < df.loc[df.index[i-1],'ub']) or 
        (df.loc[df.index[i-1],'close'] > df.loc[df.index[i-1],'ub'])):
            df.loc[df.index[i],'ub'] = df.loc[df.index[i],'basic_ub']
        else:
            df.loc[df.index[i],'ub'] = df.loc[df.index[i-1],'ub']
        
        if ((df.loc[df.index[i],'basic_lb'] > df.loc[df.index[i-1],'lb']) or 
        (df.loc[df.index[i-1],'close'] < df.loc[df.index[i-1],'ub'])):
            df.loc[df.index[i],'lb'] = df.loc[df.index[i],'basic_lb']
        else:
            df.loc[df.index[i],'lb'] = df.loc[df.index[i-1],'lb']

    isUpTrend,isDownTrend = 'up','down'
    init_trend_dir = isDownTrend
    df['trend_direction'] = init_trend_dir
    df['super_trend'] = pd.Series(dtype=float)
    for i in range(len(df)):
        if i < n:
            df.loc[df.index[i], 'trend_direction'] = init_trend_dir
        else:
            prev_super_trend = df.loc[df.index[i-1], 'super_trend'] if i > n else None
            prev_upper_band = df.loc[df.index[i-1], 'ub']
            prev_lower_band = df.loc[df.index[i-1], 'lb']
            
            if prev_super_trend == prev_upper_band:
                if df.loc[df.index[i], 'close'] > df.loc[df.index[i], 'ub']:
                    df.loc[df.index[i], 'trend_direction'] = isUpTrend
                else:
                    df.loc[df.index[i], 'trend_direction'] = isDownTrend
            else:
                if df.loc[df.index[i], 'close'] < df.loc[df.index[i], 'lb']:
                    df.loc[df.index[i], 'trend_direction'] = isDownTrend
                else:
                    df.loc[df.index[i], 'trend_direction'] = isUpTrend
            
            if df.loc[df.index[i], 'trend_direction'] == isUpTrend:
                df.loc[df.index[i], 'super_trend'] = df.loc[df.index[i], 'lb']
            else:
                df.loc[df.index[i], 'super_trend'] = df.loc[df.index[i], 'ub']

    return df

In [27]:
new_df = compute_super_trend(new_df,7,3)
new_df

,open,high,low,close,volume,hl2,tr,atr,basic_ub,basic_lb,ub,lb,trend_direction,super_trend
date,,,,,,,,,,,,,,
2024-06-03 09:15:00+05:30,1440.00,1440.00,1422.50,1426.05,608537,1431.250,NaN,NaN,NaN,NaN,NaN,NaN,down,NaN
2024-06-03 09:20:00+05:30,1425.35,1426.70,1418.45,1421.45,334479,1422.575,8.25,NaN,NaN,NaN,NaN,NaN,down,NaN
2024-06-03 09:25:00+05:30,1421.85,1422.25,1415.75,1418.85,150139,1419.000,6.50,NaN,NaN,NaN,NaN,NaN,down,NaN
2024-06-03 09:30:00+05:30,1419.00,1419.15,1413.05,1414.50,153774,1416.100,6.10,NaN,NaN,NaN,NaN,NaN,down,NaN
2024-06-03 09:35:00+05:30,1414.55,1415.05,1410.60,1410.80,197865,1412.825,4.45,NaN,NaN,NaN,NaN,NaN,down,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-07 13:30:00+05:30,1531.30,1532.10,1530.00,1531.10,154639,1531.050,2.10,2.798700,1539.446099,1522.653901,1539.446099,1522.653901,up,1522.653901
2024-06-07 13:35:00+05:30,1531.10,1534.45,1531.00,1533.35,129119,1532.725,3.45,2.891742,1541.400227,1524.049773,1539.446099,1524.049773,up,1524.049773
2024-06-07 13:40:00+05:30,1533.35,1534.10,1531.60,1532.70,218945,1532.850,2.50,2.835779,1541.357338,1524.342662,1539.446099,1524.342662,up,1524.342662
